Deep learning prediction of images from the CIFAR image dataset using convolutional neural network. Image classification of the malaria-Tensorflow dataset using a pretrained MobileNetV2 model

@author feBueno, June 2020 fernando.bueno.gutie@gmail.com

In [3]:
%tensorflow_version 2.x 
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

Load, split and normalize data

In [4]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()# Load data 

train_images, test_images = train_images / 255.0, test_images / 255.0# Normalize pixel values to be between 0 and 1

class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

Data exploration

In [ ]:
print("train_images class: "+ str(type(train_images)))
print("train_images shape: "+ str(train_images.shape))# input shape
print("one pixel input: "+ str(train_images[0,4,17]))# each pixel is a vector of length 3
print("Labels of the firts 10 samples: "+str(train_labels[:10])) # print

In [ ]:
IMG_INDEX = 0  # Image index

plt.imshow(train_images[IMG_INDEX] ,cmap=plt.cm.binary)
plt.xlabel(class_names[train_labels[IMG_INDEX][0]])
plt.show()

Model building

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))# Each image will be divided in 32 filters of 3x3 pixels
model.add(layers.MaxPooling2D((2, 2)))# Stride of 2 (overlap between filter-maps) and 2x2pooling (number of pixels for similarity-assesment)
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

print(model.summary()) 

#adding dense layers
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10))

print("model after adding dense layers: ")
print(model.summary())

Model training

In [8]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),#the targets are the category indexes. More efficient than CategoricalCrossentropy for large number of categories.
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=4, 
                    validation_data=(test_images, test_labels))

Epoch 1/4
1563/1563 [==============================] - 72s 46ms/step - loss: 1.5327 - accuracy: 0.4391 - val_loss: 1.2747 - val_accuracy: 0.5460
Epoch 2/4
1563/1563 [==============================] - 69s 44ms/step - loss: 1.1636 - accuracy: 0.5870 - val_loss: 1.0828 - val_accuracy: 0.6228
Epoch 3/4
1563/1563 [==============================] - 69s 44ms/step - loss: 1.0181 - accuracy: 0.6408 - val_loss: 1.0397 - val_accuracy: 0.6291
Epoch 4/4
1563/1563 [==============================] - 68s 43ms/step - loss: 0.9243 - accuracy: 0.6777 - val_loss: 0.9631 - val_accuracy: 0.6611


Model evaluation

In [9]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print(test_acc)# ~65% accuracy 

313/313 - 4s - loss: 0.9631 - accuracy: 0.6611
0.6610999703407288


Data augmentation

In [ ]:
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator

# create data generator to transform
datagen = ImageDataGenerator(
featurewise_std_normalization=True,
samplewise_std_normalization=True,
rotation_range=20,
width_shift_range=0.1,
height_shift_range=0.1,
shear_range=0.3,
zoom_range=0.2,
vertical_flip=True,
fill_mode='wrap')#Points outside the boundaries of the input are filled with repeats 

test_img = train_images[0]# pick an image to be augmented
img = image.img_to_array(test_img)  # convert to numpy array
img = img.reshape((1,) + img.shape)  # reshape: change shape without changing data

i = 0

for batch in datagen.flow(img, save_prefix='dataAugmentationTest', save_format='jpeg'):  # save images in current directory
    plt.figure(i)
    plot = plt.imshow(image.img_to_array(batch[0]))
    i += 1
    if i > 4:  # show the first 4 images
        break

plt.show()

Using pretrained models

In [11]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
keras = tf.keras

Load and split data

In [12]:
import tensorflow_datasets as tfds
tfds.disable_progress_bar()

# split the data manually into 80% training, 10% testing, 10% validation
(raw_train, raw_validation, raw_test), metadata = tfds.load(
    'malaria',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],#80% for train, 10% for validation, 10% for test
    with_info=True,
    as_supervised=True,
)

In [ ]:
get_label_name = metadata.features['label'].int2str  # creates a function object that we can use to get labels

# display 4 images from the dataset
for image, label in raw_train.take(4):
  plt.figure()
  plt.imshow(image)
  plt.title(get_label_name(label))

Data preprocessing

In [14]:
#standarize image size
def format_example(image, label, img_size = 160):
  """
  returns an image that is reshaped to img_size x img_size
  """
  image = tf.cast(image, tf.float32)
  image = (image/127.5) - 1# 255/2=127.5
  image = tf.image.resize(image, (img_size, img_size))
  return image, label

In [15]:
train = raw_train.map(format_example)
validation = raw_validation.map(format_example)
test = raw_test.map(format_example)

In [ ]:
for image, label in train.take(2):
  plt.figure()
  plt.imshow(image)
  plt.title(get_label_name(label))

Define data batchs

In [17]:
batch_size = 30000#the dataset will take the first 30000 filenames and put them in its buffer, then pick one at random among them. 
#since the malaria dataset contains 27,558 samples, any image can be selected

train_batches = train.batch(batch_size)
validation_batches = validation.batch(batch_size)
test_batches = test.batch(batch_size)

In [ ]:
#show image-size prior and after resizing
for img, label in raw_train.take(2):
  print("Original shape:", img.shape)

for img, label in train.take(2):
  print("New shape:", img.shape)

Picking a pretrained model

In [ ]:
img_shape = (160, 160, 3)

# Create the base model from the pre-trained model MobileNet V2. This was dessigned for 1000 classes problems
base_model = tf.keras.applications.MobileNetV2(input_shape=img_shape,
                                               include_top=False,# do not include the classifier
                                               weights='imagenet')# pre-training on ImageNet
base_model.summary()

The pretrained model outputs (32, 5, 5, 1280) that is a feauture extraction of our original (1,160, 160, 3) 

In [ ]:
base_model.trainable = False
base_model.summary()

Adding a classifier for Malaria VS Healthy

In [21]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

In [22]:
prediction_layer = keras.layers.Dense(1)# 1 single dense neuron because there are only two classes to predict

In [23]:
model = tf.keras.Sequential([
  base_model,
  global_average_layer,
  prediction_layer
])

In [24]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_160 (Model) (None, 5, 5, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 1281      
Total params: 2,259,265
Trainable params: 1,281
Non-trainable params: 2,257,984
_________________________________________________________________


Model training

In [25]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),# from logits for numeric stability
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_batches,
                    epochs=initial_epochs,
                    validation_data=validation_batches)

acc = history.history['accuracy']
print(acc)

In [ ]:
model.save("malaria.h5")
new_model = tf.keras.models.load_model('malaria.h5')